Player Stats
Data Science Process:
1) Formulate a data science question:
Predict the number of points, assists, and rebounds a position might get.
Use location, back to back factor, opponent's average number of points conceded from each position.

In [573]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import player_position_scraper
import plotly.express as px
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, teamgamelog, playergamelog, boxscoretraditionalv2, leaguegamefinder, LeagueGameFinder, playbyplayv2
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [546]:
player_position_scraper.player_pos_extracts()
def player_stats(player_name):
    player = players.find_players_by_full_name(player_name)[0]
    player_id = player['id']
    game_log = playergamelog.PlayerGameLog(player_id=player_id, season='2024-25', season_type_all_star='Regular Season')
    game_log_df = game_log.get_data_frames()[0]
    

    # Print the game log
    return game_log_df


Summary Statistics

Average number of points, rebounds and assists

In [1173]:
player_df = player_stats("Dillon Brooks")
player_df[['PTS', 'AST', 'REB']].describe()

,PTS,AST,REB
count,17.000000,17.000000,17.000000
mean,12.529412,1.294118,3.529412
std,6.539473,1.104802,1.662740
min,2.000000,0.000000,1.000000
25%,7.000000,0.000000,2.000000
50%,14.000000,1.000000,4.000000
75%,17.000000,2.000000,4.000000
max,28.000000,3.000000,7.000000


In [1174]:
player_df['GAME_DATE'] = pd.to_datetime(player_df['GAME_DATE'])
player_df['Location'] = np.where(player_df['MATCHUP'].str.contains('vs'), 'Home', 'Away')
player_df['days_diff'] = player_df['GAME_DATE'].diff(periods=-1).dt.days
player_df = player_df.rename(columns={'Game_ID': 'GAME_ID'})
player_df

,SEASON_ID,Player_ID,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Location,days_diff
0,22024,1628415,0022400031,2024-11-22,HOU vs. POR,W,25,10,15,0.667,6,8,0.750,2,4,0.50,3,3,6,1,0,0,2,3,28,13,1,Home,2.0
1,22024,1628415,0022400254,2024-11-20,HOU vs. IND,W,22,1,10,0.100,0,7,0.000,2,2,1.00,1,0,1,3,0,1,1,2,4,8,1,Home,2.0
2,22024,1628415,0022400248,2024-11-18,HOU @ MIL,L,21,2,8,0.250,1,3,0.333,0,0,0.00,2,2,4,1,1,0,1,5,5,-3,1,Away,1.0
3,22024,1628415,0022400241,2024-11-17,HOU @ CHI,W,28,7,13,0.538,1,3,0.333,0,1,0.00,0,3,3,3,2,0,1,1,15,29,1,Away,2.0
4,22024,1628415,0022400016,2024-11-15,HOU vs. LAC,W,33,6,12,0.500,5,9,0.556,0,0,0.00,0,1,1,2,0,0,2,5,17,37,1,Home,2.0
5,22024,1628415,0022400222,2024-11-13,HOU vs. LAC,W,30,5,9,0.556,4,6,0.667,1,2,0.50,1,4,5,2,0,0,1,3,15,6,1,Home,2.0
6,22024,1628415,0022400213,2024-11-11,HOU vs. WAS,W,33,4,12,0.333,0,2,0.000,0,0,0.00,2,2,4,3,2,0,2,1,8,20,1,Home,1.0
7,22024,1628415,0022400201,2024-11-10,HOU @ DET,W,34,6,15,0.400,2,8,0.250,0,0,0.00,0,2,2,1,1,0,0,1,14,14,1,Away,2.0
8,22024,1628415,0022400192,2024-11-08,HOU @ OKC,L,27,6,12,0.500,3,4,0.750,2,2,1.00,2,2,4,1,0,0,1,3,17,-11,1,Away,2.0
9,22024,1628415,0022400173,2024-11-06,HOU vs. SAS,W,25,4,9,0.444,1,3,0.333,1,1,1.00,0,2,2,0,1,0,0,2,10,10,1,Home,2.0


Average points as days_diff increases

In [1175]:
avg_points_per_diff = player_df.groupby(['days_diff','Location'])['PTS'].mean().reset_index()

# Create a scatter plot with Plotly
# fig = px.scatter(avg_points_per_diff, x='days_diff', y='PTS', color='Location',
#                  title='Average Points Scored vs Days Between Games (Home vs Away)',
#                  labels={'days_diff': 'Days Between Games', 'points_scored': 'Average Points Scored'},
#                  symbol='Location',  # Different symbols for Home and Away
#                  trendline='ols'  # Optional: Adds a trendline
# )
fig = px.bar(avg_points_per_diff, x='days_diff', y='PTS', color='Location',
                 title='Average Points Scored vs Days Between Games (Home vs Away)',
                 labels={'days_diff': 'Days Between Games', 'points_scored': 'Average Points Scored'},
)

# Show the plot
fig.show()

In [1176]:
avg_rebounds_per_diff = player_df.groupby(['days_diff','Location'])['REB'].mean().reset_index()

# Create a scatter plot with Plotly
# fig = px.scatter(avg_rebounds_per_diff, x='days_diff', y='PTS', color='Location',
#                  title='Average rebounds Scored vs Days Between Games (Home vs Away)',
#                  labels={'days_diff': 'Days Between Games', 'rebounds_scored': 'Average rebounds Scored'},
#                  symbol='Location',  # Different symbols for Home and Away
#                  trendline='ols'  # Optional: Adds a trendline
# )
fig = px.bar(avg_rebounds_per_diff, x='days_diff', y='REB', color='Location',
                 title='Average rebounds Scored vs Days Between Games (Home vs Away)',
                 labels={'days_diff': 'Days Between Games', 'rebounds_scored': 'Average rebounds Scored'},
)

# Show the plot
fig.show()

In [1177]:
nba_teams = teams.get_teams()

# Create a dictionary with team ID as the key and team name as the value
team_id_name_dict = {team['full_name']: team['id'] for team in nba_teams}

print(team_id_name_dict)

{'Atlanta Hawks': 1610612737, 'Boston Celtics': 1610612738, 'Cleveland Cavaliers': 1610612739, 'New Orleans Pelicans': 1610612740, 'Chicago Bulls': 1610612741, 'Dallas Mavericks': 1610612742, 'Denver Nuggets': 1610612743, 'Golden State Warriors': 1610612744, 'Houston Rockets': 1610612745, 'Los Angeles Clippers': 1610612746, 'Los Angeles Lakers': 1610612747, 'Miami Heat': 1610612748, 'Milwaukee Bucks': 1610612749, 'Minnesota Timberwolves': 1610612750, 'Brooklyn Nets': 1610612751, 'New York Knicks': 1610612752, 'Orlando Magic': 1610612753, 'Indiana Pacers': 1610612754, 'Philadelphia 76ers': 1610612755, 'Phoenix Suns': 1610612756, 'Portland Trail Blazers': 1610612757, 'Sacramento Kings': 1610612758, 'San Antonio Spurs': 1610612759, 'Oklahoma City Thunder': 1610612760, 'Toronto Raptors': 1610612761, 'Utah Jazz': 1610612762, 'Memphis Grizzlies': 1610612763, 'Washington Wizards': 1610612764, 'Detroit Pistons': 1610612765, 'Charlotte Hornets': 1610612766}


In [1189]:
team_name = 'Houston Rockets'
target_team_id = team_id_name_dict[team_name] #Select the team

# Step 1: Find recent games involving the target team
# Using leaguegamefinder to get recent game IDs involving the target team
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=target_team_id)
games = gamefinder.get_data_frames()[0]

# Extract the game for this season only
games_df = games[games["GAME_DATE"] >= '2024-10-22']
game_ids = games_df['GAME_ID'].tolist()
#games_df = games[['GAME_ID', 'GAME_DATE', 'MATCHUP']]
games_df['Location'] = ['home' if 'vs' in matchup else 'away' for matchup in games_df['MATCHUP']]
games_df['Opponents'] = [matchup[len(matchup)-4:] for matchup in games_df['MATCHUP']]

/var/folders/vm/thttpdpx7dxcn7xqs0lp98dsfl_3b_/T/ipykernel_78540/4176003539.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/vm/thttpdpx7dxcn7xqs0lp98dsfl_3b_/T/ipykernel_78540/4176003539.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1190]:
#Overall Game Stats
#avg_points_per_location_conceded = games_df.groupby(['Location'])['PTS'].mean().reset_index()

# Create a scatter plot with Plotly
# fig = px.bar(avg_points_per_location_conceded, x='Location', y='PTS', color='Location',
#                 barmode='group',
#                  title='Average Points Scored based on location)',
#                  labels={'PTS': 'Average Points Scored'},
# 
average_y = games_df['PTS'].mean()
# average_y_home = games_df[games_df['Location'] == 'home']['PTS'].mean()
# average_y_away = games_df[games_df['Location'] == 'away']['PTS'].mean()
fig = px.line(games_df, x='GAME_DATE', y='PTS', color='Location', title=f'Total Number of Points Scored Over Time by {team_name}', text='Opponents', markers = True)
fig.add_hline(y=average_y, line_dash="dash", line_color="black", annotation_text=f"Overall Average:{average_y}", annotation_position="top left")
# fig.add_hline(y=average_y_home, line_dash="dash", line_color="blue", annotation_text="Home Average", annotation_position="top right")
# fig.add_hline(y=average_y_away, line_dash="dash", line_color="orange", annotation_text="Away Average", annotation_position="bottom right")
fig.update_yaxes(range=[0, 150])

# Show the plot
fig.show()

In [1191]:
opponent_boxscore = []

for game_id in game_ids:
    # Retrieve box score data for each game
    boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
    boxscore_df = boxscore.get_data_frames()[0]
    
    # Filter box score data to keep only opponents of the target team
    opponent_boxscore.append(boxscore_df[boxscore_df['TEAM_ID'] != target_team_id])
    

# Concatenate all box scores into a single DataFrame
box_scores_df = pd.concat(opponent_boxscore, ignore_index=True)

player_pos_df = pd.read_csv('player_positions.csv')
merge_df = pd.merge(box_scores_df, player_pos_df, on = "PLAYER_NAME", how='left')
games_gameid_df = games_df[['GAME_ID', 'GAME_DATE', 'MATCHUP','Location', 'Opponents']]
opponents_df = pd.merge(merge_df, games_gameid_df, on = "GAME_ID", how = "left")
opponents_df = opponents_df[opponents_df['Location'].notna()] #Removes the preseason games as we are merging with player data which does not have preseason data
opponents_df['MIN'] = opponents_df['MIN'].str.split(':').str[0]
opponents_df_more_20_mins = opponents_df[opponents_df['MIN'].astype(float) > 20] #Selecting players who played at least 20 minutes

In [1192]:
#Opponents points conceded over time
total_points_per_location_conceded = opponents_df.groupby(['Location','GAME_DATE', 'TEAM_ABBREVIATION'])['PTS'].sum().reset_index()
average_y = total_points_per_location_conceded['PTS'].mean()
fig = px.line(total_points_per_location_conceded, x='GAME_DATE', y='PTS', color='Location', 
                title=f'Total Number of Points Conceded by {team_name} Over Time',text='TEAM_ABBREVIATION', markers = True)
fig.add_hline(y=average_y, line_dash="dash", line_color="black", annotation_text=f"Overall Average:{average_y}", annotation_position="top left")
fig.update_yaxes(range=[0, 150])

In [1193]:
avg_points_per_pos_conceded = opponents_df_more_20_mins.groupby(['Position','Location'])['PTS'].mean().reset_index()

# Create a scatter plot with Plotly
fig = px.bar(avg_points_per_pos_conceded, x='Position', y='PTS', color='Location',
                barmode='group',
                 title=f'Average Points Scored by Position (Home vs Away) against {team_name}',
                 labels={'Position': 'Player Position', 'PTS': 'Average Points Scored'},
)

# Show the plot
fig.show()
opponents_df_more_20_mins.groupby('Position')['PTS'].describe()

,count,mean,std,min,25%,50%,75%,max
Position,,,,,,,,
C,13.0,15.307692,8.250097,6.0,9.0,14.0,17.0,29.0
PF,18.0,12.555556,7.106105,2.0,6.5,11.0,19.5,23.0
PG,21.0,15.857143,9.911033,0.0,8.0,16.0,24.0,34.0
SF,19.0,10.052632,4.428113,2.0,7.0,11.0,13.0,16.0
SG,31.0,11.935484,6.185658,4.0,8.0,10.0,14.0,29.0


Adding a note here to add another metric that shows where a player ranks among other players in similar position on average. For example if 75% of players score at least 13 points for a specific position, compare the average points of those players in 25% and the player you want to bet on. If the player averages higher than the 25%, it means you should bet on the player.

In [1194]:
avg_assists_per_pos_conceded = opponents_df_more_20_mins.groupby(['Position','Location'])['AST'].mean().reset_index()

# Create a scatter plot with Plotly
fig = px.bar(avg_assists_per_pos_conceded, x='Position', y='AST', color='Location',
                barmode='group',
                 title=f'Average Assists Scored by Position (Home vs Away)against {team_name}',
                 labels={'Position': 'Player Position', 'AST': 'Average Assists Scored'},
)

# Show the plot
fig.show()
opponents_df_more_20_mins.groupby('Position')['AST'].describe()

,count,mean,std,min,25%,50%,75%,max
Position,,,,,,,,
C,13.0,1.230769,1.300887,0.0,1.0,1.0,1.0,5.0
PF,18.0,2.111111,1.567647,0.0,1.0,2.0,3.0,5.0
PG,21.0,5.000000,3.346640,0.0,2.0,4.0,8.0,11.0
SF,19.0,0.894737,0.875261,0.0,0.0,1.0,1.0,3.0
SG,31.0,2.741935,2.112940,0.0,1.0,3.0,4.0,7.0


In [1195]:
avg_rebounds_per_pos_conceded = opponents_df_more_20_mins.groupby(['Position','Location'])['REB'].mean().reset_index()

# Create a scatter plot with Plotly
fig = px.bar(avg_rebounds_per_pos_conceded, x='Position', y='REB', color='Location',
                barmode='group',
                 title=f'Average Rebounds Scored by Position (Home vs Away) against {team_name}',
                 labels={'Position': 'Player Position', 'REB': 'Average Rebounds Scored'},
)

# Show the plot
fig.show()
opponents_df_more_20_mins.groupby('Position')['REB'].describe()

,count,mean,std,min,25%,50%,75%,max
Position,,,,,,,,
C,13.0,9.538462,4.754215,5.0,6.0,8.0,10.0,20.0
PF,18.0,6.555556,3.293884,3.0,4.0,6.0,9.0,13.0
PG,21.0,3.571429,2.785165,0.0,1.0,3.0,6.0,9.0
SF,19.0,4.052632,2.247806,1.0,2.5,4.0,5.5,8.0
SG,31.0,3.935484,1.998924,1.0,2.0,4.0,5.0,8.0


In [1196]:
avg_3pa_per_pos_conceded = opponents_df_more_20_mins.groupby(['Position','Location'])['FG3A'].mean().reset_index()

# Create a scatter plot with Plotly
fig = px.bar(avg_3pa_per_pos_conceded, x='Position', y='FG3A', color='Location',
                barmode='group',
                 title=f'Average Three Points Attempted by Position (Home vs Away) against {team_name}',
                 labels={'Position': 'Player Position', 'FG3A': 'Average Three Point Attempts'},
)

# Show the plot
fig.show()
opponents_df_more_20_mins.groupby('Position')['FG3A'].describe()

,count,mean,std,min,25%,50%,75%,max
Position,,,,,,,,
C,13.0,2.769231,2.891189,0.0,0.0,2.0,5.00,8.0
PF,18.0,3.000000,2.473388,0.0,1.0,2.5,3.75,9.0
PG,21.0,5.095238,2.547641,2.0,4.0,4.0,6.00,12.0
SF,19.0,4.210526,2.175085,1.0,3.0,4.0,5.50,9.0
SG,31.0,4.193548,2.166708,1.0,3.0,4.0,5.00,10.0


In [1197]:
#opponents_df_more_20_mins_more_20_mins[(opponents_df_more_20_mins_more_20_mins['Location'] == 'home') & (opponents_df_more_20_mins_more_20_mins['Position'] == 'PG')]
#opponents_df_more_20_mins[(opponents_df_more_20_mins['Location'] == 'home') & (opponents_df_more_20_mins['Position'] == 'SG')]
opponents_df_more_20_mins[(opponents_df_more_20_mins['Position'] == 'C')]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,Unnamed: 0,Position,GAME_DATE,MATCHUP,Location,Opponents
17,0022400254,1610612754,IND,Indiana,1626167,Myles Turner,Myles,C,,28.000000,3.0,7.0,0.429,0.0,1.0,0.000,4.0,5.0,0.800,0.0,9.0,9.0,2.0,0.0,5.0,4.0,3.0,10.0,-13.0,69.0,C,2024-11-20,HOU vs. IND,home,IND
30,0022400248,1610612749,MIL,Milwaukee,201572,Brook Lopez,Brook,C,,39.000000,11.0,15.0,0.733,5.0,8.0,0.625,0.0,1.0,0.000,1.0,9.0,10.0,1.0,3.0,4.0,0.0,1.0,27.0,14.0,97.0,C,2024-11-18,HOU @ MIL,away,MIL
45,0022400241,1610612741,CHI,Chicago,202696,Nikola Vučević,Nikola,C,,25.000000,3.0,7.0,0.429,1.0,3.0,0.333,2.0,2.0,1.000,0.0,6.0,6.0,2.0,0.0,0.0,2.0,2.0,9.0,-35.0,25.0,C,2024-11-17,HOU @ CHI,away,CHI
59,0022400016,1610612746,LAC,LA,1627826,Ivica Zubac,Ivica,C,,29.000000,2.0,7.0,0.286,0.0,0.0,0.000,2.0,2.0,1.000,3.0,8.0,11.0,0.0,0.0,1.0,2.0,4.0,6.0,-15.0,58.0,C,2024-11-15,HOU vs. LAC,home,LAC
72,0022400222,1610612746,LAC,LA,1627826,Ivica Zubac,Ivica,C,,27.000000,4.0,8.0,0.500,0.0,0.0,0.000,0.0,2.0,0.000,2.0,4.0,6.0,1.0,1.0,1.0,0.0,2.0,8.0,-1.0,58.0,C,2024-11-13,HOU vs. LAC,home,LAC
98,0022400201,1610612765,DET,Detroit,1630191,Isaiah Stewart,Isaiah,C,,33.000000,7.0,9.0,0.778,0.0,0.0,0.000,2.0,2.0,1.000,5.0,3.0,8.0,1.0,1.0,2.0,1.0,4.0,16.0,2.0,190.0,C,2024-11-10,HOU @ DET,away,DET
110,0022400192,1610612760,OKC,Oklahoma City,1631096,Chet Holmgren,Chet,C,,29.000000,8.0,14.0,0.571,4.0,6.0,0.667,9.0,10.0,0.900,1.0,4.0,5.0,1.0,0.0,3.0,1.0,1.0,29.0,16.0,121.0,C,2024-11-08,HOU @ OKC,away,OKC
122,0022400173,1610612759,SAS,San Antonio,1641705,Victor Wembanyama,Victor,C,,24.000000,6.0,14.0,0.429,1.0,6.0,0.167,2.0,2.0,1.000,1.0,5.0,6.0,0.0,1.0,3.0,4.0,0.0,15.0,-18.0,34.0,C,2024-11-06,HOU vs. SAS,home,SAS
137,0022400161,1610612752,NYK,New York,1626157,Karl-Anthony Towns,Karl-Anthony,C,,32.000000,7.0,17.0,0.412,0.0,2.0,0.000,3.0,4.0,0.750,9.0,10.0,19.0,0.0,0.0,1.0,3.0,4.0,17.0,-26.0,14.0,C,2024-11-04,HOU vs. NYK,home,NYK
176,0022400109,1610612759,SAS,San Antonio,1641705,Victor Wembanyama,Victor,C,,35.000000,6.0,14.0,0.429,1.0,5.0,0.200,1.0,2.0,0.500,5.0,15.0,20.0,5.0,0.0,4.0,4.0,3.0,14.0,-7.0,34.0,C,2024-10-28,HOU @ SAS,away,SAS


In [1198]:
##Random Forest Model
#Predict the number of points a position will score 
rf_df = opponents_df_more_20_mins.dropna(subset=['Position'])
X = rf_df[['Position','Location','Opponents']]
y = rf_df[['PTS']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = pd.get_dummies(X_train, drop_first=True)
X_test =  pd.get_dummies(X_test, drop_first=True)


#push to the mongo db how well the model does based on date, teams and position

In [1199]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 72.67727742783528
R-squared: -0.3014844888221102


/var/folders/vm/thttpdpx7dxcn7xqs0lp98dsfl_3b_/T/ipykernel_78540/3075726021.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

